In [11]:
import re
import numpy as np
import zipfile
import pandas as pd
import glob
import sys
import os

- pulp fction
- acsent è
- Garçon,
- ♪♪ ["MISIRLOU"] change that to [sound]
- make the segmanted file

In [12]:
def srt_to_csv(movie_data, filename, destination):
    df = pd.DataFrame(movie_data, columns=['dialog'])
    #print(df.head())
    #print(df.columns)

    def cleansing(df, n, x):
        def remove_blank_lines(df):
            df.replace("", np.nan, inplace=True)
            df = df.dropna()
            return df

        def remove_brackets(df):
            df = df.replace(r'^-?\(.*\)$', '', regex=True)
            df = df.replace(r'\(.*?\)', '', regex=True)
            return df

        def strip_all_values(df):
            df = df.apply(lambda x: x.str.strip())
            return df

        def remove_specialchars(df):
            df = df.replace(r'[\=\-\{\}\[\]\^\#]', '', regex=True)
            df = df.replace(r'\\... ', '', regex=True)
            df = df.replace(r'<.*?>', '', regex=True)
            df['dialog'] = df['dialog'].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))
            return df

        def remove_nonchar_lines(df):
            df = df.drop(df[df.dialog.str.contains(r'^\d?\d?\d?\d|\d\d:\d\d:\d\d,.*', regex=True)].index)
            return df

        def remove_first_n_lines(df, n):
            df = df.iloc[n:]
            return df
        
        

        df = remove_blank_lines(df)
        df = strip_all_values(df)
        df = remove_brackets(df)
        df = remove_nonchar_lines(df)
        df = remove_specialchars(df)
        df = remove_blank_lines(df)
        df = remove_first_n_lines(df, n)
        df = remove_blank_lines(df)

        return df


    df_movie = cleansing(df, 1, 0)

    movie_txt = df_movie.dialog.values.tolist()

    sent = ''
    new_l = []
    for t in movie_txt:
        if t[-1] not in ['.', '?', '!']:
            sent += t + ' '
        else:
            sent += t
            new_l.append(sent)
            sent = ''
    df_movie_updated = pd.DataFrame(columns=['dialog'])
    df_movie_updated['dialog'] = new_l
    df_movie_updated.head()

    #print("converting to csv")
    df_movie_updated.to_csv(destination + filename[4:-4]+'.csv')

def main(path, destination):
    for filename in glob.glob(path+'*.srt'):
        #print(filename)
        file_encoding = 'cp1252' 
        with open(filename, encoding=file_encoding, errors='replace') as f:
            lines = f.readlines()
        new_lines = srt_to_csv(lines, filename, destination)

In [14]:
path = 'SRT/'
destination = 'CSV/'
main(path, destination)